# DiffDock with Docker for Local Development and Cloud Deployment

---

This documentation provides a guide on DiffDock implemented in Python, designed for both local development and cloud deployment using Docker. It covers the following topics:

1. **Introduction to DiffDock**: Overview of the DiffDock and its applications.
2. **Setting Up the Development Environment**: Step-by-step instructions for setting up a local development environment using Docker.
3. **Building and Running the Docker Container**: Instructions for building the Docker image and running the container.
4. **Deploying to the Cloud**: Guidelines for deploying the DiffDock to a cloud platform using Docker.
5. **Best Practices**: Tips and best practices for working with DiffDock and Docker.

## Introduction to DiffDock and the Protein-Folding Journey

---

Biology’s grand challenge has long been predicting how a linear chain of amino acids folds into its unique three-dimensional structure—a concept first crystallized by Christian Anfinsen’s thermodynamic hypothesis in the 1950s, which asserted that a protein’s sequence alone determines its native conformation ([profiles.nlm.nih.gov][1])([Wikipedia][2]).  In the decades that followed, experimental methods like X-ray crystallography and NMR dominated structural biology, but they were laborious and costly.

In the 1990s, David Baker’s group transformed the field by introducing **Rosetta**, the first large‐scale *in silico* ab initio folding algorithm that combined fragment assembly with energy functions derived from physics and statistics ([HHMI][3])([docs.rosettacommons.org][4]).  Rosetta’s success in CASP (the Critical Assessment of Structure Prediction) competitions demonstrated that computational models could rival, and eventually guide, experimental structure determination.

A new era dawned in 2020 when DeepMind’s **AlphaFold2** achieved atomic-level accuracy across CASP14 targets, reporting a median Cα RMSD of 0.96 Å—an order‐of‐magnitude leap over prior methods ([Nature][5])([Google DeepMind][6]).  Almost in tandem, the Baker lab released **RoseTTAFold**, an efficient three-track network that further democratized high-accuracy predictions ([Baker Lab][7])([Science][8]).  Meta’s **ESMFold** soon followed, leveraging large protein language models to predict structures at unprecedented scale ([Science][9])([Meta AI][10]).

While these advances excel at *predicting* known sequences, designing entirely new proteins demands **generative** approaches.  Inspired by successes in image and language generation, the Baker lab recently introduced **RFdiffusion**, which fine-tunes a RoseTTAFold network to act as a denoising diffusion model over backbone coordinates ([Nature][11])([Protein Design Institute][12]).  RFdiffusion can scaffold enzyme active sites, assemble symmetric oligomers, and create novel binding proteins with orders-of-magnitude fewer experimental tests required ([Baker Lab][13])([Baker Lab][14]).



## Notebook Roadmap

---

### Sections
- [Building and Running the Docker Container](#building-and-running-the-docker-container)
- [Using DiffDock](#using-diffdock)
- [Small GSK3B-FRAT1 Study](#small-gsk3b-frat1-study)
- [Deploying to the Cloud](#deploying-to-the-cloud)


### Prerequisites

Before you begin, ensure you have the following installed on your local machine:

- Docker: [Install Docker](https://docs.docker.com/get-docker/)
- A compatible GPU (for DiffDock)
- NVIDIA drivers (if using GPU)


## Building and Running the Docker Container

---

To build and run the Docker container for DiffDock, follow these steps:

1. **Clone the Repository**: Clone the DiffDock repository to your local machine.

```bash
git clone https://github.com/gabenavarro/MLContainerLab.git
cd MLContainerLab
```

2. **Build the Docker Image**: Use the provided Dockerfile to build the Docker image.

```bash
# You can choose any tag you want for the image
# Feel free to play around with the base image, just make sure the host has the same or higher CUDA version
docker build -f ./assets/build/Dockerfile.diffdock.cu117cp310 -t diffdock:117-310 .
```
3. **Run the Docker Container**: Run the Docker container with the necessary configurations. In the first example, we will run the container locally with GPU support. This is the recommended way to run a container while in development mode. For scaling up, we will use the second example which runs the container in the cloud.

```bash
   # Run the container with GPU support
   docker run -dt \
      --gpus all \
      --shm-size=64g \
      -v "$(pwd):/workspace" \
      --name diffdock \
      --env NVIDIA_VISIBLE_DEVICES=all \
      --env GOOGLE_APPLICATION_CREDENTIALS=/workspace/assets/secrets/gcp-key.json \
      diffdock:126-310
   ```
> Note: The `-v "$(pwd):/workspace"` option mounts the current directory to `/workspace` in the container, allowing you to access your local files from within the container. The `--env` options set environment variables for GPU visibility and Google Cloud credentials.<br>
> Note: The `--gpus all` option allows the container to use all available GPUs. <br>

4. **Access the Container with IDE**: In this example, we will use Visual Studio Code to access the container. You can use any IDE of your choice.

```bash
# In a scriptable manner
CONTAINER_NAME=diffdock
FOLDER=/workspace
HEX_CONFIG=$(printf {\"containerName\":\"/$CONTAINER_NAME\"} | od -A n -t x1 | tr -d '[\n\t ]')
code --folder-uri "vscode-remote://attached-container+$HEX_CONFIG$FOLDER"
```

> Note: The `code` command is used to open Visual Studio Code. Make sure you have the Remote - Containers extension installed in VS Code to access the container directly. <br>
> Note: Make sure you have installed Remote - Containers extension in VS Code.<br>



[1]: https://profiles.nlm.nih.gov/spotlight/kk/feature/protein "Protein Folding and the Thermodynamic Hypothesis, 1950-1962"
[2]: https://en.wikipedia.org/wiki/Anfinsen%27s_dogma "Anfinsen's dogma - Wikipedia"
[3]: https://www.hhmi.org/news/rosetta-may-hold-key-predicting-protein-folding "Rosetta May Hold Key to Predicting Protein Folding - HHMI"
[4]: https://docs.rosettacommons.org/docs/latest/meta/Rosetta-Timeline "History of Rosetta"
[5]: https://www.nature.com/articles/s41586-021-03819-2 "Highly accurate protein structure prediction with AlphaFold - Nature"
[6]: https://deepmind.google/discover/blog/alphafold-using-ai-for-scientific-discovery-2020/ "AlphaFold: Using AI for scientific discovery - Google DeepMind"
[7]: https://www.bakerlab.org/publications/ "Publications - Baker Lab"
[8]: https://www.science.org/doi/10.1126/science.abj8754 "Accurate prediction of protein structures and interactions using a ..."
[9]: https://www.science.org/doi/10.1126/science.ade2574 "Evolutionary-scale prediction of atomic-level protein structure with a ..."
[10]: https://ai.meta.com/blog/protein-folding-esmfold-metagenomics/ "ESM Metagenomic Atlas: The first view of the 'dark matter ... - Meta AI"
[11]: https://www.nature.com/articles/s41586-023-06415-8 "De novo design of protein structure and function with RFdiffusion"
[12]: https://www.ipd.uw.edu/2022/12/a-diffusion-model-for-protein-design/ "A diffusion model for protein design"
[13]: https://www.bakerlab.org/2023/07/11/diffusion-model-for-protein-design/ "RFdiffusion: A generative model for protein design - Baker Lab"
[14]: https://www.bakerlab.org/2023/03/30/rf-diffusion-now-free-and-open-source/ "RFdiffusion now free and open source - Baker Lab"
